In [1]:
import functools

import gymnasium
import numpy as np
from gymnasium.spaces import Discrete

from pettingzoo import AECEnv
from pettingzoo.utils import agent_selector, wrappers

In [2]:
ROCK = 0
PAPER = 1
SCISSORS = 2
NONE = 3
MOVES = ["ROCK", "PAPER", "SCISSORS", "None"]
NUM_ITERS = 5  # 这个常量是最大时间步。原为100。
REWARD_MAP = {
    (ROCK, ROCK): (0, 0),
    (ROCK, PAPER): (-1, 1),
    (ROCK, SCISSORS): (1, -1),
    (PAPER, ROCK): (1, -1),
    (PAPER, PAPER): (0, 0),
    (PAPER, SCISSORS): (-1, 1),
    (SCISSORS, ROCK): (-1, 1),
    (SCISSORS, PAPER): (1, -1),
    (SCISSORS, SCISSORS): (0, 0),
}

In [4]:
class raw_env(AECEnv):
    metedata = {"render_modes": ["human"], "name": "rps_v2"}

    def __init__(self, render_mode=None):
        self.possible_agents = ["player_" + str(r) for r in range(2)]
        self.agent_name_mapping = dict(
            zip(self.possible_agents, list(range(len(self.possible_agents))))
        )
        self._action_spaces = {agent: Discrete(3) for agent in self.possible_agents}
        self._observation_spaces = {
            agent: Discrete(4) for agent in self.possible_agents
        }
        self.render_mode = render_mode

    def reset(self, seed=None, return_info=False, options=None):
        self.agents = self.possible_agents[:]
        self.rewards = {agent: 0 for agent in self.agents}
        self._cumulative_rewards = {agent: 0 for agent in self.agents}
        self.terminations = {agent: False for agent in self.agents}
        self.truncations = {agent: False for agent in self.agents}
        self.infos = {agent: {} for agent in self.agents}
        self.state = {agent: NONE for agent in self.agents}
        self.observations = {agent: NONE for agent in self.agents}
        self.num_moves = 0
        self._agent_selector = agent_selector(self.agents)
        self.agent_selection = self._agent_selector.next()

    @functools.lru_cache(maxsize=None)
    def observation_space(self, agent):
        return Discrete(4)

    @functools.lru_cache(maxsize=None)
    def action_space(self, agent):
        return Discrete(3)

    def render(self):
        if self.render_mode is None:
            gymnasium.logger.warn(
                "You are calling render method without specifying any render mode."
            )
            return
        if len(self.agents) == 2:
            string = "Current state: Agent1: {} , Agent2: {}".format(
                MOVES[self.state[self.agents[0]]], MOVES[self.state[self.agents[1]]]
            )
        else:
            string = "Game over"
        print(string)

    def observe(self, agent):
        return np.array(self.obsevations)

    def close(self):
        pass

    def step(self, action):
        if (
                self.terminations[self.agent_selection] or self.truncations[self.agent_selection]
        ):
            self._was_dead_step(action)
            return
        agent = self.agent_selection
        self._cumulative_rewards[agent] = 0
        self.state[self.agent_selection] = action


In [3]:
a = ["player_" + str(r) for r in range(2)]
b = dict(zip(a, list(range(len(a)))))
print(a)
print(b)

['player_0', 'player_1']
{'player_0': 0, 'player_1': 1}


In [3]:
x = 2
if 3 <= x <= 4:
    print("right")

In [ ]:
class Solution(object):
    def checkOverlap(self, radius, xCenter, yCenter, x1, y1, x2, y2):
        """
        :type radius: int
        :type xCenter: int
        :type yCenter: int
        :type x1: int
        :type y1: int
        :type x2: int
        :type y2: int
        :rtype: bool
        """
        self.radius = radius
        self.xCenter = xCenter
        self.yCenter = yCenter
        self.x1 = x1
        self.y1 = y1
        self.x2 = x2
        self.y2 = y2
        self.xsquare = 0.5 * (self.x1 + self.x2)
        self.ysquare = 0.5 * (self.y1 + self.y2)
        self.k = (self.ysquare - self.yCenter) / (self.xsquare - self.xCenter)
        dot_list = list()
        in_circle = False
        in_square = False
        if x1 <= xCenter <= x2 and y1 <= yCenter <= y2:
            return True
        xsquare = 0.5 * (x1 + x2)
        ysquare = 0.5 * (y1 + y2)
        tmp = 0
        dot1 = (x1, self.y(x1))
        if y1 <= dot1[1] <= y2:
            dot_list.append(dot1)
        dot2 = (x2, self.y(x2))
        if y1 <= dot2[1] <= y2:
            dot_list.append(dot2)
        dot3 = (self.x(y1), y1)
        if x1 <= dot3 <= x2:
            dot_list.append(dot3)
        dot4 = (self.x(y2), y2)
        if x1 <= dot4 <= x2:
            dot_list.append(dot4)
        for i in dot_list:
            if (i[0] - xCenter) ** 2 + (i[1] - yCenter) ** 2 <= radius ** 2:
                return True
        return False

    def y(self, x):
        return self.k * (x - self.xsquare) + self.ysquare

    def x(self, y):
        return 1 / self.k * (y - self.ysquare) + self.xsquare